In [ ]:
#!pip install flask
#!pip install tensorflow
#!pip install tensorflow_hub
#!pip install pickle
#!pip install bz2
import sys
sys.path.append('C:\\Users\\hp\\AppData\\Local\\Programs\\Python\\Python37\\Lib\\site-packages')
from flask import Flask, render_template, request
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import pickle
import bz2

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [299, 299])

def get_predictions(test_data,threshold=None):
    """
    Returns predictions for binary classification given `threshold`
    For instance, if threshold is 0.3, then it'll output 1 (malignant) for that sample if
    the probability of 1 is 30% or more (instead of 50%)
    """
    prediction=""
    # load the raw data from the file as a string
    img = tf.io.read_file(test_data)
    img = decode_img(img)
    plt.figure(figsize=(12,12))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(img[n])
        plt.axis('off')
    X_test = np.zeros((1, 299, 299, 3))
    for i, (img) in enumerate(img):
        X_test[0] = img
    # building the model
    # InceptionV3 model & pre-trained weights
    module_url = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"
    m = tf.keras.Sequential([hub.KerasLayer(module_url, output_shape=[2048], trainable=False),tf.keras.layers.Dense(1, activation="sigmoid")])

    m.build([None, 299, 299, 3])
    m.load_weights("benign-vs-malignant_64_rmsprop_0.432.h5")
    #load trained model
    y_pred = m.predict(X_test)
    print(y_pred)
    if not threshold:
        threshold = 0.5
    result = ""
    # test melanoma probability
    if y_pred[0] <= threshold:
        result = "Cancerous Skin Lesions \n Malignant Tissue Detected "
    else:
        result = "Non-Cancerous Skin Lesion \n Benign Tissues Detected"
    return result
#create an instance of Flask

@app.route('/predict', methods = ['POST','GET'])
def predict():
    Image = request.form['img']
    #put all inputs in array
    test_data = 'Sample/'+Image
    print(test_data)
    threshold = 0.52   
    #predict
    r= get_predictions(test_data,threshold) 
    print(r)
    return render_template('index.html', prediction=r)
    pass

@app.route('/doc', methods = ['POST','GET'])
def doc():
     return render_template('doc.html')

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [01/Nov/2022 22:14:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2022 22:14:49] "GET /css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [01/Nov/2022 22:14:49] "GET /static/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Nov/2022 22:14:49] "GET /css/pogo-slider.min.css HTTP/1.1" 404 -
127.0.0.1 - - [01/Nov/2022 22:14:49] "GET /css/responsive.css HTTP/1.1" 404 -
127.0.0.1 - - [01/Nov/2022 22:14:49] "GET /static/css/responsive.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Nov/2022 22:14:49] "GET /static/css/custom.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Nov/2022 22:14:49] "GET /css/style.css HTTP/1.1" 404 -
127.0.0.1 - - [01/Nov/2022 22:14:49] "GET /static/css/pogo-slider.min.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Nov/2022 22:14:49] "GET /static/js/bootstrap.min.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Nov/2022 22:14:49] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Nov/2022 22:14:49] "GET /css/custom.cs

Sample/ISIC_0012204.jpg


ERROR:__main__:Exception on /predict [POST]
Traceback (most recent call last):
  File "C:\Users\hp\AppData\Local\Programs\Python\Python37\Lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\hp\AppData\Local\Programs\Python\Python37\Lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\hp\AppData\Local\Programs\Python\Python37\Lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\hp\AppData\Local\Programs\Python\Python37\Lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_13516\3973316204.py", line 77, in predict
    r= get_predictions(test_data,threshold)
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_13516\3973316204.py", line 51, in get_predicti